月次集計(タイムゾーンが JST, Asia/Taipei ”以外”の SSP)

※第二営業日に回す

In [3]:
import pandas as pd
import numpy as np
import os

import pathlib
import zipfile

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

#from src import impala_config_hc2 as h2
#from src import impala_config_hc3 as h3

In [2]:
#Gsheet URL
url = "https://docs.google.com/spreadsheets/d/1S_-N_caMj6iMOPNJW8WwwYQV18-eoXEHTwv-YR3DLyM/edit#gid=202356699"

#シート上のデータ消去
for i in range(1,30):
    gs = GSheet()
    gs.clear_sheets(url, [i])

10.Pubmatic (PST8PDT, USD)

In [4]:
query = (
    f"""
    SELECT
      year_local,
      month_local,
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as day_local,
        CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint) / currency_rate AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 10
        AND
      action_type = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
     currency_rate IS NOT NULL
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local,
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as day_local,
        CAST(winning_price AS bigint) / currency_rate AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 10
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      currency_rate IS NOT NULL
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    gs = GSheet()
    gs.add_data(url, 21, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

11.AppNexus (EST5EDT, USD) AppNexus 全部

In [5]:
query = (
    f"""
    SELECT
      year_local,
      month_local, 
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
       coalesce(
              original_winning_price,
              winning_price) / currency_rate AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 11
        AND
      action_type = "I"
        AND
      (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local, 
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
        winning_price / currency_rate AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 11
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
      (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;

    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url,5, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

11.AppNexus (EST5EDT, USD) AppNexus MAX のみ

In [6]:
query = (
    f"""
    SELECT
      year_local,
      month_local, 
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
       coalesce(
              original_winning_price,
              winning_price) / currency_rate AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 11
        AND
      action_type = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      seller_member_id = 280
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local, 
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
        winning_price / currency_rate AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 11
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      seller_member_id = 280
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 8, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

11.AppNexus (EST5EDT, USD) AppNexus Kiosked のみ

In [7]:
query = (
    f"""
    SELECT
      year_local,
      month_local, 
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
       coalesce(
              original_winning_price,
              winning_price) / currency_rate AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 11
        AND
      action_type = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      seller_member_id = 2721
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local, 
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
        winning_price / currency_rate AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 11
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      seller_member_id = 2721
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;

    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 6, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

11.AppNexus (EST5EDT, USD) AppNexus YDN のみ

In [8]:
query = (
    f"""
      SELECT
      year_local,
      month_local, 
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
       coalesce(
              original_winning_price,
              winning_price) / currency_rate AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 11
        AND
      action_type = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      seller_member_id = 3663
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local, 
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
        winning_price / currency_rate AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 11
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      seller_member_id = 3663
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 7, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

11.AppNexus (EST5EDT, USD) AppNexusその他のみ ※その他 = MAX, Kiosked, YDN 以外

In [9]:
query = (
    f"""
        SELECT
      year_local,
      month_local, 
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
       coalesce(
              original_winning_price,
              winning_price) / currency_rate AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 11
        AND
      action_type = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      seller_member_id NOT IN (280, 2721, 3663)
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local, 
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
        winning_price / currency_rate AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 11
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      seller_member_id NOT IN (280, 2721, 3663)
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;

    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 9, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

16.Rubicon Project (PST8PDT, USD)

In [10]:
query = (
    f"""
    SELECT
      year_local,
      month_local, 
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as day_local,
        CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint) / currency_rate AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 16
        AND
      action_type = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local, 
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as day_local,
        CAST(winning_price AS bigint) / currency_rate AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 16
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 22, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

22.Brightroll (EST5EDT, USD)

In [11]:
query = (
    f"""
    SELECT
  year_local,
  month_local,
  day_local,
  "OK" AS "Result",
  COUNT(1)  AS "Imp",
  SUM(spend) / 1000 AS "Spend (USD)"
FROM (
  SELECT
    year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
    month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
    day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
   coalesce(
          original_winning_price,
          winning_price) / currency_rate AS spend
  FROM dsplog.adserver
  WHERE
    ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
       OR
     (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
    AND
  ssp_id      = 22
    AND
  action_type = "I"
    AND
 (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
  ) AS local
WHERE
  year_local = year(add_months(now(), -1))
    AND
  month_local = month(add_months(now(), -1))
GROUP BY year_local, month_local, day_local
 
UNION ALL
 
SELECT
  year_local,
  month_local,
  day_local,
  "Delayed" AS "Result",
  COUNT(1)  AS "Imp",
  SUM(spend) / 1000 AS "Spend (USD)"
FROM (
  SELECT
    year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as year_local,
    month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as month_local,
    day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'EST5EDT')) as day_local,
    winning_price / currency_rate AS spend
  FROM dsplog.adserver_ilg
  WHERE
    ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
       OR
     (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
    AND
  ssp_id      = 22
    AND
  action_type = "I"
    AND
  result_type = "TOE"
    AND
 (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
  ) AS local
WHERE
  year_local = year(add_months(now(), -1))
    AND
  month_local = month(add_months(now(), -1))
GROUP BY year_local, month_local, day_local;

    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 11, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

26.Tead (JST, USD)

In [12]:
query = (
    f"""
    SELECT
      year,
      month,
      day,
      "OK"     AS "Result",
      COUNT(1) AS "Imp",
      SUM(CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint) / currency_rate) / 1000 AS "Spend (USD)"
    FROM dsplog.adserver
    WHERE
      year         = cast(year(add_months(now(), -1)) as string)
        AND
      month        = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id       = 26
        AND
      action_type  = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day

    UNION ALL

    SELECT
      year,
      month,
      day,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(CAST(winning_price AS bigint) / currency_rate) / 1000 AS "Spend (USD)"
    FROM dsplog.adserver_ilg
    WHERE
      year        = cast(year(add_months(now(), -1)) as string)
        AND
      month       = lpad(cast(month(add_months(now(), -1)) as string), 2, "0")
        AND
      ssp_id      = 26
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
        AND
      dsp_company_id = 1
    GROUP BY year, month, day;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 24, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

29.Tenmax (USD, Asia/Taipei)

In [13]:
query = (
    f"""
    SELECT
      year_local,
      month_local,
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as day_local,
       coalesce(
              original_winning_price,
              winning_price) / currency_rate AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 29
        AND
      action_type = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local,
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'Asia/Taipei')) as day_local,
        winning_price / currency_rate AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 29
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 25, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

30.Outbrain(USD, JST) ※ JST は仮，確定したらクエリの `interval 0 hours` を変更

In [14]:
query = (
    f"""
    SELECT
      year_local,
      month_local,
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 0 hours, 'Asia/Tokyo')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 0 hours, 'Asia/Tokyo')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 0 hours, 'Asia/Tokyo')) as day_local,
       coalesce(
              original_winning_price,
              winning_price) / currency_rate AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 30
        AND
      action_type = "I"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local,
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 0 hours, 'Asia/Tokyo')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 0 hours, 'Asia/Tokyo')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 0 hours, 'Asia/Tokyo')) as day_local,
        winning_price / currency_rate AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 30
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
     (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 27, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

10. Pubmatic USD移行後 (PST8PDT, JPY)  2018/12/5-

query = (
    f"""
SELECT
  year_local,
  month_local,
  day_local,
  "OK" AS "Result",
  COUNT(1)  AS "Imp",
  SUM(spend) / 1000 AS "Spend (USD)"
FROM (
  SELECT
    year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as year_local,
    month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as month_local,
    day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as day_local,
    CAST(coalesce(
          original_winning_price,
          winning_price) AS bigint) / currency_rate AS spend
  FROM dsplog.adserver
  WHERE
    ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
       OR
     (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
    AND
  ssp_id      = 10
    AND
  action_type = "I"
    AND
 (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    AND
 currency_rate IS NOT NULL
  ) AS local
WHERE
  year_local = year(add_months(now(), -1))
    AND
  month_local = month(add_months(now(), -1))
GROUP BY year_local, month_local, day_local
  
UNION ALL
  
SELECT
  year_local,
  month_local,
  day_local,
  "Delayed" AS "Result",
  COUNT(1)  AS "Imp",
  SUM(spend) / 1000 AS "Spend (USD)"
FROM (
  SELECT
    year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as year_local,
    month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as month_local,
    day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as day_local,
    CAST(winning_price AS bigint) / currency_rate AS spend
  FROM dsplog.adserver_ilg
  WHERE
    ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
       OR
     (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
    AND
  ssp_id      = 10
    AND
  action_type = "I"
    AND
  result_type = "TOE"
    AND
 (smn_uid is null or smn_uid not in (1, 1111111111111, 2222222222222)) and (test IS NULL OR test != TRUE)
    AND
  currency_rate IS NOT NULL
  ) AS local
WHERE
  year_local = year(add_months(now(), -1))
    AND
  month_local = month(add_months(now(), -1))
GROUP BY year_local, month_local, day_local;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 21, df, [0, 1], True)

11. Mobub  2019/3-

In [15]:
query = (
    f"""
    SELECT
      year_local,
      month_local,
      day_local,
      "OK" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as day_local,
        CAST(coalesce(
              original_winning_price,
              winning_price) AS bigint) / currency_rate AS spend
      FROM dsplog.adserver
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 33
        AND
      action_type = "I"
        AND
     currency_rate IS NOT NULL
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local

    UNION ALL

    SELECT
      year_local,
      month_local,
      day_local,
      "Delayed" AS "Result",
      COUNT(1)  AS "Imp",
      SUM(spend) / 1000 AS "Spend (USD)"
    FROM (
      SELECT
        year(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as year_local,
        month(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as month_local,
        day(from_utc_timestamp(cast(from_unixtime(unix_timestamp(messenger_time, 'yyyy-MM-dd HH:mm:ss')) as timestamp) - interval 9 hours, 'PST8PDT')) as day_local,
        CAST(winning_price AS bigint) / currency_rate AS spend
      FROM dsplog.adserver_ilg
      WHERE
        ((year = cast(year(add_months(now(), -1)) as string) and month = lpad(cast(month(add_months(now(), -1)) as string), 2, "0"))
           OR
         (year = cast(year(now()) as string) and month = lpad(cast(month(now()) as string), 2, "0") and day = "01"))
        AND
      ssp_id      = 33
        AND
      action_type = "I"
        AND
      result_type = "TOE"
        AND
      currency_rate IS NOT NULL
      ) AS local
    WHERE
      year_local = year(add_months(now(), -1))
        AND
      month_local = month(add_months(now(), -1))
    GROUP BY year_local, month_local, day_local;
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df = ir.sql_to_pandas(query)
    df['spend (usd)'] = df['spend (usd)'].astype(float)
    gs = GSheet()
    gs.add_data(url, 30, df, [0, 1], True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    S

In [16]:
import datetime
import pathlib
import math

from dateutil.relativedelta import relativedelta

In [17]:
year = datetime.date.today().year
month = datetime.date.today().month
today = datetime.date.today()
pre_month = today - relativedelta(months=1)
pre_month = datetime.date.strftime(pre_month, '%m')

In [18]:
print(pre_month)

10


In [19]:
print(year)

2019


In [ ]:
gm = GMail()


mail_body = (
    f"""

    みなさま、
    
    お疲れ様です。
    前月分の国外SSP実績（imp・spend）送ります。
    ご査証下さい。
    {url}
    
    よろしくお願い致します。   
    
    """
)

atr = gm.send_gmail(subject=f'SSP spned実績集計 {year}/{month} (国外SSP分)', 
                    message_text=mail_body,
                    to_addrs=['lsakaguchi@so-netmedia.jp','planning@so-netmedia.jp'],
                    cc_addrs=['dm@so-netmedia.jp','yusuke_akada@so-netmedia.jp'])